In [1]:
import copy
import json
import os

import boto3
import requests
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest
from dotenv import load_dotenv
from fastmcp import Client
from rich.pretty import pprint

load_dotenv()

True

### Let's implement and test the simplest flow
1. User asks a question
2. Agent decides which tool to use
3. Agent uses the tool
4. Agent returns the answer to the user

### Let's define the tools

In [2]:
mcp_client = Client("02_agent_mcp_server.py")
await mcp_client._connect()

mcp_tools = await mcp_client.list_tools()

tools = [
        {
            "name": tool.name,
            "description": tool.description,
            "input_schema": getattr(tool, 'inputSchema', {"type": "object", "properties": {}}),
        }
        for tool in mcp_tools
    ]
# pprint(mcp_tools, expand_all=True)
pprint(tools, expand_all=True)

[
│   {
│   │   'name': 'get_weather',
│   │   'description': 'Get the current weather in a given location\n\nArgs:\n    latitude: Latitude of the location for which to get the weather\n    longitude: Longitude of the location for which to get the weather\n\nReturns:\n    JSON with current weather information',
│   │   'input_schema': {
│   │   │   'properties': {
│   │   │   │   'latitude': {
│   │   │   │   │   'type': 'string'
│   │   │   │   },
│   │   │   │   'longitude': {
│   │   │   │   │   'type': 'string'
│   │   │   │   }
│   │   │   },
│   │   │   'required': [
│   │   │   │   'latitude',
│   │   │   │   'longitude'
│   │   │   ],
│   │   │   'type': 'object'
│   │   }
│   },
│   {
│   │   'name': 'facts_from_wikipedia',
│   │   'description': 'Get Wikipedia facts about a given topic, location, person etc.\n\nArgs:\n    topic: A topic, location, person etc. to search for on Wikipedia\n\nReturns:\n    JSON with Wikipedia information',
│   │   'input_schema': {
│   │   │   'properties': {
│   │   │   │   'topic': {
│   │   │   │   │   'type': 'string'
│   │   │   │   }
│   │   │   },
│   │   │   'required': [
│   │   │   │   'topic'
│   │   │   ],
│   │   │   'type': 'object'
│   │   }
│   }
]

### Let's define the user's message

In [3]:
user_message = "What's the weather in Warsaw? Is there anything interesting about this city?"

### Let's make the first LLM call to decide which tool to use and with what parameters

In [4]:
session = boto3.Session()
credentials = session.get_credentials()

def llm_call(messages: list[dict]) -> dict:
    region = os.getenv("AWS_REGION", "eu-central-1")
    url = f"https://bedrock-runtime.{region}.amazonaws.com/model/eu.anthropic.claude-sonnet-4-5-20250929-v1:0/invoke"

    llm_headers = {
        "Content-Type": "application/json",
    }
    payload = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 300,
        "tools": tools,
        "tool_choice": {"type": "auto"},
        "system": """You are an assistant that responds to user question based on the tools used. Respond politely to user's question using the output from the tools. Use Celsius scale. If the tools output is empty, then always respond with "I don't know". Use only information from the tools output to answer the question.""",
        "messages": messages,
    }
    print_llm_call(messages)
    request = AWSRequest(method='POST', url=url, data=json.dumps(payload), headers=llm_headers)
    SigV4Auth(credentials, "bedrock", region).add_auth(request)
    resp = requests.post(url, headers=dict(request.headers), data=json.dumps(payload))
    return resp.json()

In [5]:
def print_llm_call(messages: list[dict]) -> None:
    messages_copy = copy.deepcopy(messages)
    # trim too long content from output of tools
    MAX_TOOL_RESULT_LEN = 1000
    for msg in messages_copy:
        if msg.get("role") == "user" and isinstance(msg.get("content"), list):
            for item in msg["content"]:
                if isinstance(item, dict) and item.get("type") == "tool_result":
                    content = item.get("content", "")
                    if isinstance(content, str) and len(content) > MAX_TOOL_RESULT_LEN:
                        item["content"] = content[:MAX_TOOL_RESULT_LEN] + "...[truncated]"
    print(f"> LLM Call: {json.dumps(messages_copy, indent=2)}")

In [6]:
llm_call([{"role": "user", "content": "use weather tool for warsaw", }])

> LLM Call: [
  {
    "role": "user",
    "content": "use weather tool for warsaw"
  }
]


{'model': 'claude-sonnet-4-5-20250929',
 'id': 'msg_bdrk_011Z3Hqq6zBccQ8YyHYawiEi',
 'type': 'message',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': "I'll get the weather information for Warsaw. Let me look up Warsaw's coordinates first, then get the weather data."},
  {'type': 'tool_use',
   'id': 'toolu_bdrk_014bNn5QKbU3GYxUYtmZmLPi',
   'name': 'get_weather',
   'input': {'latitude': '52.2297', 'longitude': '21.0122'}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 793,
  'cache_creation_input_tokens': 0,
  'cache_read_input_tokens': 0,
  'output_tokens': 101}}

In [7]:
messages = [{"role": "user", "content": user_message}]

response = llm_call(messages)
i = 1
while response['stop_reason'] == "tool_use":
    print(f"---------------------")
    print(f"Interation {i}")
    tool_results = []
    assistant_content = []

    for block in response['content']:
        if block['type'] == "text":
            assistant_content.append(block)
        elif block['type'] == "tool_use":
            print(f"> Tool: {block['name']}")
            print(f"  Input: {json.dumps(block['input'], indent=2)}")

            result = await mcp_client.call_tool(block['name'], block['input'])
            if hasattr(result, 'content') and result.content:
                result_str = result.content[0].text
            else:
                result_str = str(result)

            assistant_content.append(block)
            tool_results.append({
                "type": "tool_result",
                "tool_use_id": block['id'],
                "content": result_str
            })

    messages.append({"role": "assistant", "content": assistant_content})
    messages.append({"role": "user", "content": tool_results})

    response = llm_call(messages)
    i+=1

final_response = ""
for block in response['content']:
    if block['type'] == "text":
        final_response += block['text']

print(f"---------------------")
print("Final Response:")
print(final_response)

> LLM Call: [
  {
    "role": "user",
    "content": "What's the weather in Warsaw? Is there anything interesting about this city?"
  }
]
---------------------
Interation 1
> Tool: get_weather
  Input: {
  "latitude": "52.2297",
  "longitude": "21.0122"
}
> Tool: facts_from_wikipedia
  Input: {
  "topic": "Warsaw"
}
> LLM Call: [
  {
    "role": "user",
    "content": "What's the weather in Warsaw? Is there anything interesting about this city?"
  },
  {
    "role": "assistant",
    "content": [
      {
        "type": "text",
        "text": "I'd be happy to help you with information about Warsaw! Let me get the current weather and some interesting facts about the city."
      },
      {
        "type": "tool_use",
        "id": "toolu_bdrk_01Y8N8oC65CxvdAaJHq4WNEo",
        "name": "get_weather",
        "input": {
          "latitude": "52.2297",
          "longitude": "21.0122"
        }
      },
      {
        "type": "tool_use",
        "id": "toolu_bdrk_01RfRgidQAYWLpmeBut8TP2e

In [8]:
# Cleanup
await mcp_client._disconnect()